# 월마트 판매

## #01. 준비작업

### [1] 패키지 참조

In [430]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(work_path)

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *

### [2] 데이터 가져오기

주요 소매업체의 매출에 영향을 미칠 수 있는 요소 찾기

https://www.kaggle.com/datasets/mikhail1681/walmart-sales


| 변수명 | 설명 |
|:--:|--:|
| Store | Store number |
| Date | Sales week start date |
| Weekly_Sales | Store week sales |
| Holiday_Flag | Mark on the presence or absence of a holiday |
| Temperature | Air temperature in the region |
| Fuel_Price | Fuel cost in the region |
| CPI | Consumer price index(소비자 물가 지수) |
| Unemployment | Unemployment rate |

In [431]:
origin = my_read_data("./res/Walmart_sales.csv", info = False)
my_pretty_table(origin.head())

+----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------+
|    |   Store | Date       |   Weekly_Sales |   Holiday_Flag |   Temperature |   Fuel_Price |     CPI |   Unemployment |
|----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------|
|  0 |       1 | 05-02-2010 |    1.64369e+06 |              0 |         42.31 |        2.572 | 211.096 |          8.106 |
|  1 |       1 | 12-02-2010 |    1.64196e+06 |              1 |         38.51 |        2.548 | 211.242 |          8.106 |
|  2 |       1 | 19-02-2010 |    1.61197e+06 |              0 |         39.93 |        2.514 | 211.289 |          8.106 |
|  3 |       1 | 26-02-2010 |    1.40973e+06 |              0 |         46.63 |        2.561 |  211.32 |          8.106 |
|  4 |       1 | 05-03-2010 |    1.55481e+06 |              0 |          46.5 |        2.625 |  211.35 |          8.106 |
+----+---------+--------

In [432]:
df = origin.copy()



In [433]:
# 주차별 평균 / 중간값

df_date = df[['Date','Weekly_Sales']].groupby('Date').median().sort_values('Weekly_Sales').reset_index()
map_date = {}
for i,v in enumerate(df_date['Date']):
    map_date[v] = df_date['Weekly_Sales'][i]
df['Date_value'] = df['Date'].map(map_date)

In [434]:
df_store = df[['Store','Weekly_Sales']].groupby('Store').median().sort_values('Weekly_Sales').reset_index()
map_store = {}
for i,v in enumerate(df_store['Store']):
    map_store[v] = df_store['Weekly_Sales'][i]
df['Store_value'] = df['Store'].map(map_store)


In [435]:
df2 = df.drop(['Store','Date'],axis=True)
df2

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value
0,1643690.90,0,42.31,2.572,211.096358,8.106,1033017.37,1534849.64
1,1641957.44,1,38.51,2.548,211.242170,8.106,1015684.09,1534849.64
2,1611968.17,0,39.93,2.514,211.289143,8.106,988467.61,1534849.64
3,1409727.59,0,46.63,2.561,211.319643,8.106,899761.48,1534849.64
4,1554806.68,0,46.50,2.625,211.350143,8.106,1005669.58,1534849.64
...,...,...,...,...,...,...,...,...
6430,713173.95,0,64.88,3.997,192.013558,8.684,928629.31,764014.06
6431,733455.07,0,64.89,3.985,192.170412,8.667,979825.92,764014.06
6432,734464.36,0,54.47,4.000,192.327265,8.667,948613.39,764014.06
6433,718125.53,0,56.47,3.969,192.330854,8.667,960945.43,764014.06


In [436]:
# my_pairplot(df2)

In [437]:
df2['Holiday_Flag'] = df2['Holiday_Flag'].astype('uint8') 
df2.dtypes

Weekly_Sales    float64
Holiday_Flag      uint8
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
Date_value      float64
Store_value     float64
dtype: object

In [438]:
temperature = []
for i in df2['Temperature']:
    temperature.append((i-32)*5/9)
df2['Temperature'] = temperature
df2.columns

Index(['Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Date_value', 'Store_value'],
      dtype='object')

In [439]:
df2['Temperature2'] = df2['Temperature']**2
df2['Temperature3'] = df2['Temperature']**3
df2['Fuel_Price2'] = df2['Fuel_Price']**2
df2['CPI2'] = df2['CPI']**2
df2['Unemployment2'] = df2['Unemployment']**2


In [440]:
df3 = my_replace_outliner_to_nan(df2, 'Weekly_Sales').dropna()
df2.describe()

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Temperature3,Fuel_Price2,CPI2,Unemployment2
count,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6.435000e+03,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,1.046965e+06,0.069930,15.924324,3.358607,171.578394,7.999151,9.713045e+05,1.024393e+06,358.572563,8691.572376,11.490906,30987.855331,67.504814
std,5.643666e+05,0.255049,10.247185,0.459020,39.356712,1.875885,1.128847e+05,5.284339e+05,314.203222,10006.412494,3.068967,13617.643212,34.769820
min,2.099862e+05,0.000000,-18.922222,2.472000,126.064000,3.879000,7.864592e+05,2.584274e+05,0.000123,-6775.111011,6.110784,15892.132096,15.046641
25%,5.533501e+05,0.000000,8.588889,2.933000,131.735000,6.891000,9.211784e+05,5.560461e+05,77.880633,633.601008,8.602489,17354.110225,47.485881
50%,9.607460e+05,0.000000,17.038889,3.445000,182.616521,7.874000,9.592291e+05,9.542339e+05,290.513086,4946.793855,11.868025,33348.793560,61.999876
75%,1.420159e+06,0.000000,23.855556,3.735000,212.743293,8.622000,9.911044e+05,1.378340e+06,569.087562,13575.901417,13.950225,45259.708921,74.338884
max,3.818686e+06,1.000000,37.855556,4.468000,227.232807,14.313000,1.682862e+06,2.073951e+06,1433.043086,54248.642171,19.963024,51634.748486,204.861969


In [441]:
my_anova(df3[['Holiday_Flag','Weekly_Sales']],hue = 'Holiday_Flag',target = 'Weekly_Sales')

pingouin.anova
+----+--------------+-------------+------+-------------+---------+-----------+-------------+
|    | Source       |          SS |   DF |          MS |       F |     p-unc |         np2 |
|----+--------------+-------------+------+-------------+---------+-----------+-------------|
|  0 | Holiday_Flag | 1.22323e+12 |    1 | 1.22323e+12 | 4.11732 | 0.0424875 | 0.000643018 |
|  1 | Within       |  1.9011e+15 | 6399 | 2.97094e+11 |     nan |       nan |         nan |
+----+--------------+-------------+------+-------------+---------+-----------+-------------+

statsmodels.anova.anova_lm
+-----------------+------+-------------+-------------+---------+-----------+
|                 |   df |      sum_sq |     mean_sq |       F |    PR(>F) |
|-----------------+------+-------------+-------------+---------+-----------|
| C(Holiday_Flag) |    1 | 1.22323e+12 | 1.22323e+12 | 4.11732 | 0.0424875 |
| Residual        | 6399 |  1.9011e+15 | 2.97094e+11 |     nan |       nan |
+-------------

In [442]:
df4 = my_standard_scaler(df3, 'Weekly_Sales')
df4.head()

,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Temperature3,Fuel_Price2,CPI2,Unemployment2,Holiday_Flag,Weekly_Sales
0,-1.002520,-1.713516,1.002474,0.055203,0.601172,0.980669,-1.041902,-0.853426,-1.589018,0.995032,-0.053358,0,1643690.90
1,-1.208867,-1.765728,1.006179,0.055203,0.438913,0.980669,-1.104687,-0.867466,-1.629003,0.999554,-0.053358,1,1641957.44
2,-1.131758,-1.839696,1.007372,0.055203,0.184137,0.980669,-1.084545,-0.863652,-1.685008,1.001011,-0.053358,0,1611968.17
3,-0.767937,-1.737446,1.008147,0.055203,-0.646249,0.980669,-0.936067,-0.818575,-1.607391,1.001957,-0.053358,0,1409727.59
4,-0.774996,-1.598213,1.008922,0.055203,0.345167,0.980669,-0.939787,-0.819992,-1.499389,1.002904,-0.053358,0,1554806.68


In [443]:
yname = 'Weekly_Sales'
x_train, x_test, y_train, y_test = my_train_test_split(df4, yname, test_size=0.2)
my_linear_regrassion(x_train, y_train, x_test, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 14713.969 * Temperature + 181596.002 * Fuel_Price + 21878.330 * CPI + -23813.585 * Unemployment + 93492.313 * Date_value + 531598.911 * Store_value + -346.272 * Temperature2 + -16087.730 * Temperature3 + -178972.089 * Fuel_Price2 + -23625.175 * CPI2 + 26500.751 * Unemployment2 + 11768.717 * Holiday_Flag + 1034953.853

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.967996 |             64824.6 |         9.59224e+09 |            97940 |                         7.65955 |              -0.11244 |
+--------+----------------+---------------------+---------------------+------------------+

LinearRegression(n_jobs=-1)

In [455]:
ignore = ['CPI2','Fuel_Price2','Temperature2','Unemployment2','CPI','Unemployment','Holiday_Flag','Fuel_Price','Temperature','Temperature3']
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 93972.518 * Date_value + 532229.681 * Store_value + 1035809.120

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.967497 |             65912.8 |         9.74188e+09 |          98700.9 |                         7.71577 |             -0.137207 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+--------------+-------------+--------------------+------------+------------------+--------+------------+---------+
|    | 종속변수     | 독립변수    |   B(비표준화 계수) |   표준오차 |   β(표준화 

LinearRegression(n_jobs=-1)